Student Name: Gerard Boland

Student ID: 19196067

****E-tivity 2 - Task 1****

Q1. Choose two of the following matrix operations and write Python functions to perform the operations.

1. Provide the size of a matrix as a 2-dimensional tuple - **DONE**
2. Sum/subtract the matrix with another matrix of suitable size - **DONE**
3. Multiply the matrix with a matrix or vector of suitable size - **Also attempted**

Q2. Choose two of the following matrix operations and write Python functions to perform the operations.

4. Calculate the determinant of a 2x2 matrix. For any other matrix this function should raise a suitable exception. - **DONE**
5. Calculate the inverse of a 2x2 matrix.  For any other matrix this function should raise a suitable exception. - **DONE**
6. Calculate the transpose of an mxn matrix. (formerly "Calculate the Eigen values of a 2x2 matrix.") For any other matrix this function should raise a suitable exception. - **I've also attempted all of these, just for my own interest**

Choose one of the following file operations and write a Python function to perform the operation:

7. Export the matrix to a CSV file - **DONE**
8. Import a matrix from CSV file - **DONE - so that I could test the above**

You may import one suitable library to help you with the CSV import/export.



**Code is a little repetitive, and I've probably gone overboard on checks :)**

In [15]:
import csv

class MatrixOps:

    @staticmethod    
    def __row_size(matrix) -> int:
        return len(matrix)

    @staticmethod
    def __column_size(matrix) -> int:
        return len(matrix[0])

    @staticmethod
    def __verify_matrix(matrix):
        # Matrix defined as list of lists (rows). Reject empty list.
        if not isinstance(matrix, list):
            raise ValueError("Matrix needs to be specified as a list of lists")

        if len(matrix) == 0:
            raise ValueError("Empty (0x0) Matrix not accepted")

        # Fail if not containing a list of lists
        # Assert the rows are not all equal sizes (or all empty).
        for row in matrix:
            if not isinstance(row, list):
                raise ValueError("Matrix needs to be specified as a list of lists")
            if len(row) == 0:
                raise ValueError("Matrix must have at least one entry on each row")
            if len(row) != len(matrix[0]):
                raise ValueError("Matrix needs to be specified as a list of lists of equal size")

            # Assert if there's any non-numeric entries.
            for entry in row:
                if not isinstance(entry, (int, float)):
                    raise ValueError("Matrix can only contain Real numbers (i.e. int or float)")

    @staticmethod
    def __verify_vector(vector):
        # Vector defined as list of numbers. Reject empty list.
        if not isinstance(vector, list):
            raise ValueError("Vector needs to be specified as a list")

        if len(vector) == 0:
            raise ValueError("Empty Vector not accepted")

        for entry in vector:
            if not isinstance(entry, (int, float)):
                raise ValueError("Vector can only contain Real numbers (i.e. int or float)")

    @staticmethod
    def __verify_scalar(scalar):
        if not isinstance(scalar, (int, float)):
              raise ValueError("Argument needs be be a number: ", scalar)

    def __size(self, matrix):
        return (self.__row_size(matrix), self.__column_size(matrix))

    def multiply_matrix_by_scalar(self, matrix, scalar:float):
        self.__verify_matrix(matrix)
        self.__verify_scalar(scalar)

        rows = self.__row_size(matrix)
        cols = self.__column_size(matrix)
        
        result = matrix # handy way to preallocate
        for i in range(0, rows):
            for j in range(0, cols):
                result[i][j] = scalar * matrix[i][j]

        return result

    def multiply_matrix_by_matrix(self, matrix_a, matrix_b):
        self.__verify_matrix(matrix_a)
        self.__verify_matrix(matrix_b)

        # A * B
        if self.__column_size(matrix_a) != self.__row_size(matrix_b):
             raise ValueError("Cannot multiply matrices with incompatible sizes:", self.size(matrix_a), self.size(matrix_b))

        rows = self.__row_size(matrix_a)
        cols = self.__column_size(matrix_b)
        crossover = self.__column_size(matrix_a)
        # Output matrix will have size (rows, cols)

        # Preallocating the result matrix makes mult algorithm cleaner on the eye in python
        result = [[ 0 for j in range(cols)  ] for i in range(rows)]

        for i in range(0, rows):
            for j in range(0, cols):
                for k in range(0, crossover):
                    result[i][j] += matrix_a[i][k] * matrix_b[k][j] 
        return result

    def multiply_vector_by_matrix(self, matrix, vector):
        self.__verify_matrix(matrix)
        self.__verify_vector(vector)

        # A * b
        if self.__column_size(matrix) != len(vector):
             raise ValueError("Cannot multiply matrix of size", self.size(matrix_a), "with vector of length", len(vector))

        result = []
        for row in matrix:
            result.append( sum([x * y for x, y in zip(row, vector)]))

        return result

    def transpose(self, matrix):
        self.__verify_matrix(matrix)

        # Result matrix will flip the row & columns
        rows = self.__column_size(matrix)
        cols = self.__row_size(matrix)

        # Preallocating the result matrix makes mult algorithm cleaner on the eye in python
        result = [[ None for j in range(cols)  ] for i in range(rows)]
        for i in range(0, rows):
            for j in range(0, cols):
                result[i][j] = matrix[j][i]

        return result

    # Return size of matrix as (row, column) tuple
    def size(self, matrix):
        self.__verify_matrix(matrix)
        return self.__size(matrix)

    def add(self, matrix_a, matrix_b):
        # A+B
        self.__verify_matrix(matrix_a)
        self.__verify_matrix(matrix_b)
        if self.__size(matrix_a) != self.__size(matrix_b):
            raise ValueError("Unable to add two matrices of different size")

        result = []
        for rows in zip(matrix_a, matrix_b):
            result.append([ sum(x) for x in zip(rows[0], rows[1]) ])

        return result

    def subtract(self, matrix_a, matrix_b):
        # A-B
        self.__verify_matrix(matrix_b)
        minus_matrix_b = self.multiply_matrix_by_scalar(matrix_b, -1)
        return self.add(matrix_a, minus_matrix_b)

    def determinant(self, matrix):
        # Determinant of a 2x2 matrix A=[[a,b],[c,d]] is ad-bc
        self.__verify_matrix(matrix)
        if self.__size(matrix) != (2, 2):
            raise ValueError("Sorry, for now this method only supports 2x2 matrices!")

        return matrix[0][0] * matrix[1][1] - (matrix[0][1] * matrix[1][0])

    def trace(self, matrix) -> float:
        # Trace of a matrix is the sum of the diagonal entries
        self.__verify_matrix(matrix)
        if self.__row_size(matrix) != self.__column_size(matrix):
            raise ValueError("Trace operation only defined for square matrices")

        result = 0
        for i in range(0, self.__row_size(matrix)):
            result += matrix[i][i]
        return result

    def eigenvalues(self, matrix):
        det = self.determinant(matrix)
        half_trace = self.trace(matrix) / 2

        d = half_trace**2 - det # if positive, 2 roots, if zero, 1 root, else no roots
        if d < 0:
            return None
        elif d == 0:
            return half_trace
        else:
            return (
                half_trace + ( half_trace**2 - det )**0.5,
                half_trace - ( half_trace**2 - det )**0.5
            )

    def inverse(self, matrix):
        det = self.determinant(matrix) # does 2x2 check too!
        if det == 0:
            raise ZeroDivisionError("Matrix cannot be inverted, has zero determinant")

        inverse = [[matrix[1][1], -matrix[0][1]],
                   [-matrix[1][0], matrix[0][0]] ]

        return self.multiply_matrix_by_scalar(inverse, 1/det);

    def import_matrix_from_csv(self, file_path:str):
        matrix = []
        # The csv module us quite primitive, it often reads in numbers as strings. Using
        # csv.QUOTE_NONNUMERIC encourages it to consider values without quotes as numbers.
        with open(file_path, 'r') as file_handle:
            reader = csv.reader(file_handle, quoting=csv.QUOTE_NONNUMERIC)
            for row in reader:
                matrix.append(row)
        self.__verify_matrix(matrix)
        return matrix
            
    def export_matrix_to_csv(self, matrix, file_path:str):
        self.__verify_matrix(matrix)
        with open(file_path, 'w', newline='') as file_handle:  # ref: https://docs.python.org/3/library/csv.html#id3
            writer = csv.writer(file_handle)
            for row in matrix:
                writer.writerow(row)

    def print(self, matrix):
        # prints a matrix slightly prettier
        self.__verify_matrix(matrix)
        print('[')
        for row in matrix:
            print(' ', row)
        print(']')

In [16]:
mat = MatrixOps()

In [17]:
# A few matrices to work on
m1 = [ [3, 6, 9], [8, 4, 2]]
m2 = [ [ 1, 2, 3], [4, 5, 6], [7, 8, 9]]
m3 = [ [ 0, 1, -1], [1, 0, -1], [-1, 1, 0]]

In [18]:
print(mat.size(m1))
print(mat.size(m2))

(2, 3)
(3, 3)


In [19]:
mat.print(mat.add(m2, m3))
mat.print(mat.subtract(m2, m3))

[
  [1, 3, 2]
  [5, 5, 5]
  [6, 9, 9]
]
[
  [1, 1, 4]
  [3, 5, 7]
  [8, 7, 9]
]


In [20]:
mat.print(mat.transpose(m1))

[
  [3, 8]
  [6, 4]
  [9, 2]
]


In [26]:
m4 = [[5, 2],[-7, -3]]

print("Determinant", mat.determinant(m4))
print("Inverse:")
mat.print(mat.inverse(m4))

print("Eigenvalues", mat.eigenvalues(m4))

Determinant -1
Inverse:
[
  [3.0, 2.0]
  [-7.0, -5.0]
]
Eigenvalues (2.414213562373095, -0.41421356237309515)


In [27]:
m6 = [ [ 0.2222, 131231341.42342, 452352525252525245245.23423423424234234], [4.2, 0.000005, 6.2], [7, 8, 9]]
mat.export_matrix_to_csv(m6, "m6.txt")

m6_copy = mat.import_matrix_from_csv("m6.txt")
mat.print(m6_copy)

[
  [0.2222, 131231341.42342, 4.523525252525252e+20]
  [4.2, 5e-06, 6.2]
  [7.0, 8.0, 9.0]
]


In [30]:
identity = [[1, 0, 0], [0, 1, 0], [0, 0, 1]]
print("Multiplying matrix by identity matrix should leave it unchanged")
m9 = mat.multiply_matrix_by_matrix(m3, identity)
mat.print(m9)
print("Test multiplication of 2 matrices")
m9 = mat.multiply_matrix_by_matrix(m2, m3)
mat.print(m9)

print("Applying Matrix to a vector")
v1 = [1 ,1, 1]
print(mat.multiply_vector_by_matrix(m2, v1))

Multiplying matrix by identity matrix should leave it unchanged
[
  [0, -1, 1]
  [-1, 0, 1]
  [1, -1, 0]
]
Test multiplication of 2 matrices
[
  [1, -4, 3]
  [1, -10, 9]
  [1, -16, 15]
]
Applying Matrix to a vector
[6, 15, 24]
